In [1]:
# install these libraries one by one
pip install numpy
pip install opencv-python
pip install keras
pip install tensorflow
pip install pillow
pip install scikit-learn
pip install pandas
pip install matplotlib

In [1]:
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten


2024-05-13 12:26:22.093611: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 12:26:22.130258: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 12:26:22.461824: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 12:26:23.691230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [3]:
# Load the trained emotion detection model
emotion_model = Sequential()
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.2))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.22))

emotion_model.add(Flatten())
emotion_model.add(Dense(512, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(256, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.load_weights("model/emotion_model.h5")

print("Loaded model weights from disk")


/home/devil/.local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-13 12:26:32.479088: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 12:26:32.479853: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and se

Loaded model weights from disk


In [9]:

# Function to detect emotion for each frame
def detect_emotion(frame):

    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 163, 250), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        emotion_prediction = emotion_model.predict(cropped_img)
        max_index = int(np.argmax(emotion_prediction))

        max_probability = emotion_prediction[0][max_index] * 100  


        emotion_label = emotion_dict[max_index]
        print(f"Predicted Emotion: {emotion_label}, Confidence: {max_probability:.2f}%")

        percentage = f"{max_probability:.2f}%"
        cv2.putText(frame, emotion_label, (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (89, 213, 224), 2, cv2.LINE_AA)
        cv2.putText(frame, percentage, (x+w-60, y-60), cv2.FONT_HERSHEY_SIMPLEX, .6, (138, 83, 224), 2, cv2.LINE_AA)



In [10]:
input_source = 'image'  # Change this to 'image', 'video', or 'camera' as needed

In [14]:

cv2.namedWindow('Emotion Detection', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Emotion Detection', 1024, 768)

if input_source == 'image':
    input_file = 'image.jpg'  # Change this to your input file path

    frame = cv2.imread(input_file)
    detect_emotion(frame)
    cv2.imshow('Emotion Detection', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

elif input_source == 'video':
    input_file = 'test.mp4'  # Change this to your input video file path
    cap = cv2.VideoCapture(input_file)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        detect_emotion(frame)
        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

elif input_source == 'camera':
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        detect_emotion(frame)
        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
else:
    print("Invalid input source.")

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread (0x830fa50).
Cannot move to target thread (0x8013e00)

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread (0x830fa50).
Cannot move to target thread (0x8013e00)

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread (0x830fa50).
Cannot move to target thread (0x8013e00)

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread (0x830fa50).
Cannot move to target thread (0x8013e00)

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread (0x830fa50).
Cannot move to target thread (0x8013e00)

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread (0x830fa50).
Cannot move to target thread (0x8013e00)

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread (0x830fa50).
Cannot move to target thread (0x8013e00)

QObject::moveToThread: Current thread (0x8013e00) is not the object's thread

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Emotion: Happy, Confidence: 99.88%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Emotion: Neutral, Confidence: 31.56%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Emotion: Surprised, Confidence: 73.18%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Emotion: Angry, Confidence: 27.63%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted Emotion: Sad, Confidence: 75.12%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Emotion: Sad, Confidence: 69.90%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicted Emotion: Sad, Confidence: 62.51%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted Emotion: Happy, Confidence: 83.97%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Emotion: Happy, Confidence: 93.93%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted Emotion: Sad, Confidence: 52.41%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted Emotion: Sad, Confidence: 99.65%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Predicted Emotion: Fearful, Confidence: 83.80%
1/1 ━━